In [39]:
import os, re
import pandas as pd
from jamo import h2j, j2h, j2hcj
from konlpy.tag import Kkma, Komoran

tokenizer = Komoran(userdic='module/userdic.txt')

In [40]:
irregular_conjugation_stems = ['걷', '긷', '깨닫', '눋', '닫', '듣', '묻', '붇', '싣', '일컫', '가깝', '가볍', '간지럽', '굽', '그립', '깁', '껄끄럽', '노엽', '더럽', '덥', '맵', '메스껍', '무겁', '반갑', '부끄럽', '사납', '서럽', '쑥스럽', '줍', '긋', '낫', '붓', '잇', '잣', '젓', '짓', ]

In [41]:
def verb_exception_handler(verb):
    if verb == '나서':
        return '나'
    if verb == '아니하':
        return '않'
    else:
        return verb

In [42]:
def verb_to_noun(verb):
    
    """
    한국어의 용언을 명사형으로 변환하는 함수입니다.
    용언(동사, 형용사)의 어간을 받아 명사형으로 반환합니다.
        
    """
    
    verb = verb_exception_handler(verb)

    eum = '음'  # 명사형 전성 어미 '음'
    m = h2j('음')[-1]  # 명사형 전성 어미 'ㅁ'

    d = h2j('ㄷ')  # ㄷ 불규칙 활용
    s = h2j('ㅅ')  # ㅅ 불규칙 활용
    b = h2j('ㅂ')  # ㅂ 불규칙 활용

    last_syllables = h2j(verb[-1])  # 용언의 마지막 자모군
    final = j2hcj(last_syllables[-1])  # 마지막 자모군의 끝글자

    pattern = re.compile(r'[ㅏ-ㅣ]')
    has_final = not bool(pattern.match(final))  # 마지막 자모군에 종성 존재 여부

    if has_final:  # 종성이 존재하는 경우
        if final == d and verb in irregular_conjugation_stems:  # ㄷ 불규칙 활용 적용
            modified = last_syllables[:-1] + h2j('ㄹ')
            return verb[:-1] + j2h(*modified) + eum
        elif final == s and verb in irregular_conjugation_stems:  # ㅅ 불규칙 활용 적용
            modified = last_syllables[:-1]
            return verb[:-1] + j2h(*modified) + eum
        if final == b and verb in irregular_conjugation_stems:  # ㅂ 불규칙 활용
            modified = h2j('우') + m
            return verb[:-1] + j2h(*last_syllables[:-1]) + j2h(*modified)
        else:  # 종성이 존재하나 불규칙 활용이 아닌 경우
            modified = last_syllables
            return verb[:-1] + j2h(*modified) + eum
    else:  # 종성이 존재하지 않는 경우
        modified = last_syllables + m
        return verb[:-1] + j2h(*modified)

# 보다 / 돌다 / 걷다 / 긋다 / 가깝다 / 반갑다 / 넘어지

# print(verb_to_noun('보'))
# print(verb_to_noun('돌'))
# print(verb_to_noun('걷'))
# print(verb_to_noun('긋'))
# print(verb_to_noun('가깝'))
# print(verb_to_noun('반갑'))
# print(verb_to_noun('넘어지'))

In [43]:
def extract_keywords(sentence):
    
    """
    문장을 입력으로 받아서 형태소 분석을 수행하고, 명사(Noun)와 동사(Verb)를 추출하는 함수
    마지막 토큰이 명사형 혹은 동사의 어근이 아닌 경우 삭제합니다.
    tokenizer별 형태소 분류표: https://docs.google.com/spreadsheets/d/1OGAjUvalBuX-oZvZ_-9tEfYD2gQe7hTGsgUpiiBSXI8/edit#gid=0

    """
    words = sentence.split()
    
    words_w_pos = []
    for word in words:
        pos = tokenizer.pos(word)
        pos = [[word, tag] for word, tag in pos]
        words_w_pos.append(pos)

    endable_pos = ['VV', 'VA', 'VX', 'NNG', 'NNP']
    poppable = False
    for word_w_pos in words_w_pos:
        for pair in word_w_pos:
            if pair[1] in endable_pos:
                poppable = True
                break
        if poppable:
            break

    if poppable:
        for idx in range(len(words_w_pos)-1, -1, -1):
            while words_w_pos[idx][-1][1] not in endable_pos:
                words_w_pos[idx].pop(-1)
                if len(words_w_pos[idx]) < 1:
                    break
            if len(words_w_pos[idx]) > 0:
                break
        words_w_pos = [word_w_pos for word_w_pos in words_w_pos if word_w_pos != []]
        return words_w_pos
    else:
        return words_w_pos

In [44]:
def join_off_syllable(test_case):
    pattern = re.compile(r'[ㄱ-ㅎ]')
    searched = pattern.search(test_case)
    
    if searched:
        off_syllable_position, *_ = searched.span()
        former_syllables = h2j(test_case[off_syllable_position-1])
        if len(former_syllables) < 3:
            off_syllable = h2j(test_case[off_syllable_position])
            modified = former_syllables + off_syllable
            modified = j2h(*modified)
            result = test_case[:off_syllable_position-1] + modified + test_case[off_syllable_position+1:]
            return result
    else:
        return test_case
    
def join_all_off_syllables(test_case):
    while True:
        result = join_off_syllable(test_case)
        if test_case == result:
            return result
        else:
            test_case = result

In [45]:
def sentence_to_noun_verb(sentence):

    """
    "사다리가 쓰러져서" 문장을 "사다리가 쓰러짐" 문자열로 변환하는 함수
    extract_keywords를 통해 얻은 keywords_w_pos에 대한 표현 정규화 작업을 수행합니다.
    리스트의 마지막 요소가 동사의 어근인 경우만 해당 작업을 수행합니다. (['VV', 'VA', 'VX'])

    """
    words_w_pos = extract_keywords(sentence)

    if words_w_pos[-1][-1][1] in ['VV', 'VA', 'VX']:
        words_w_pos[-1][-1][0] = verb_to_noun(words_w_pos[-1][-1][0])
    
    result = [[word for word, tag in word_w_pos] for word_w_pos in words_w_pos]
    result = [''.join(el) for el in result]
    result = ' '.join(result)
    
    return  join_all_off_syllables(result)

In [46]:
# test_cases = ['발판이 쓰러져서', '사다리가 넘어지다', '사다리가 넘어져서', '사다리가 넘어져', '발판이 쓰러져', '화재 발생', '배탈이 나서', '배탈이 났다']

# for item in test_cases:
#     result = sentence_to_noun_verb(item)
#     print(item, '>>>', result)
#     print()

In [47]:
# test_cases = ['결속하지 아니하여', '결속하지 않아' , '착용하지 아니한 채로', '발생하지 않은', '보지 못하고', '보지 않고', '튀어 올라',  '들어가려다가', '결속하지 않아서', '키스토링에 걸리게', '발판이 넘어져서', '발판으로 넘어져서']

# for item in test_cases:
#     result = sentence_to_noun_verb(item)
#     print(item, '>>>', result)
#     print()

### Main()

In [48]:
DATA_PATH = input('인과 객체 엑셀 파일이 존재하는 폴더명을 입력하세요\n')
print('표현 정규화 작업을 시작합니다.')

sheets = sorted([os.path.join(DATA_PATH, sheet) for sheet in os.listdir(DATA_PATH)])
col_num_count = lambda sheet: len(pd.read_excel(sheet).columns)
col_nums = [col_num_count(sheet) for sheet in sheets]

dfs = [pd.read_excel(sheet, names=[f'Col {i}' for i in range(col_num)]) for sheet, col_num in zip(sheets, col_nums)]
ce_chains_dfs = [df.iloc[:, 1:] for df in dfs]

In [49]:
ce_chains_regularized_dfs = [ce_chains.applymap(lambda x: sentence_to_noun_verb(x) if type(x) == str else x) for ce_chains in ce_chains_dfs]

In [51]:
for sheet, ce_chains_df in zip(sheets, ce_chains_dfs):
    file_name = sheet.split('/')[-1].split('.')[0]
    ce_chains_df.to_csv(f'./ce_obj_reg_results/{file_name}_original.csv', index=False, encoding='utf-8-sig')
    
for sheet, ce_chains_regularized_df in zip(sheets, ce_chains_regularized_dfs):
    file_name = sheet.split('/')[-1].split('.')[0]
    ce_chains_regularized_df.to_csv(f'./ce_obj_reg_results/{file_name}_regularized.csv', index=False, encoding='utf-8-sig')

In [10]:
# sheet_1 = 'data/eshc 인과관계 학습용(sample)_rev2.xlsx'
# sheet_2 = 'data/port 인과관계 학습용(sample)_rev4.xlsx'
# data_1 = pd.read_excel(sheet_1, names=[f'Col {i}' for i in range(10)])
# data_2 = pd.read_excel(sheet_2, names=[f'Col {i}' for i in range(9)])

In [11]:
# ce_chains_1 = data_1.iloc[:, 1:]
# ce_chains_2 = data_2.iloc[:, 1:]

In [12]:
# ce_chains_1_regularized = ce_chains_1.applymap(lambda x: sentence_to_noun_verb(x) if type(x) == str else x)
# ce_chains_2_regularized = ce_chains_2.applymap(lambda x: sentence_to_noun_verb(x) if type(x) == str else x)

In [13]:
# original = ce_chains_1.values.tolist() + ce_chains_2.values.tolist()
# regularized = ce_chains_1_regularized.values.tolist() + ce_chains_2_regularized.values.tolist()